# S1 JPEG, JPEG2000 and FFMpeg

In [1]:
!pip install pywavelets

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\22edu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import ffmpeg
from PIL import Image
from collections import OrderedDict
import pywt
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct



2) Start a script called first_seminar.py . Then create a class and a method , which is a translator from 3 values in RGB into the 3 YUV values, plus the opposite operation.

In [5]:
def RGB_to_YUV(rgb):
  m = np.array([[ 0.29900, -0.16874,  0.50000], [0.58700, -0.33126, -0.41869], [ 0.11400, 0.50000, -0.08131]])

  yuv = np.dot(rgb,m)
  yuv[:,:,1:]+=128.0

  return yuv

def YUV_to_RGB(yuv):
      
  m = np.array([[ 1.0, 1.0, 1.0], [-0.000007154783816076815, -0.3441331386566162, 1.7720025777816772], [ 1.4019975662231445, -0.7141380310058594 , 0.00001542569043522235]])
  
  rgb = np.dot(yuv,m)
  rgb[:,:,0]-=179.45477266423404
  rgb[:,:,1]+=135.45870971679688
  rgb[:,:,2]-=226.8183044444304
  return rgb

In [6]:
RGB_to_YUV(np.random.randint(0, 256, (100, 100, 3)))
YUV_to_RGB(np.random.randint(0, 256, (100, 100, 3)))

array([[[ -19.19911269,  223.38578558,  -38.48177457],
        [ 176.77357373,  182.25927353,  308.40796444],
        [ 404.21965358,  167.65935206,  269.49372518],
        ...,
        [ 146.39175932,  142.81189227,  214.02002852],
        [ 205.38503672,  -26.86055017,  257.5177001 ],
        [ 183.30284189,  321.75021052,   73.02698379]],

       [[ 279.41541519,   35.38710141,  207.74178197],
        [ 171.72876005,   80.34420776, -144.81534271],
        [ 325.60167549,  108.08659434,  203.49358635],
        ...,
        [  53.3521228 ,  107.23651099,   71.05160658],
        [ 151.58764255,  146.38990498,  252.14404149],
        [ 239.71716239,   81.48494172,  328.88516016]],

       [[  27.33276245,  131.97334123,  -93.34076936],
        [ 207.21542246,  163.44451094,  334.21632447],
        [-158.03327165,  110.89040422, -103.86596463],
        ...,
        [ 324.64174129,   70.30996394,  146.14185362],
        [  31.95926165,   36.42811537,  254.91997504],
        [ 215.88396933

3) Use ffmpeg to resize images into lower quality. Use any image you like

In [7]:
def resize_image(input_image, output_image, width, height):
    ffmpeg.input(input_image).output(output_image, vf=f'scale={width}:{height}').run(overwrite_output=True)

In [8]:
resize_image('image.png', 'image_resized.png', 50, 70)

4) Create a method called serpentine which should be able to read the bytes of a JPEG file in the serpentine way we saw.

In [ ]:
def serpentine(img):
    rows, cols = img.shape
    output = []
    resolution = rows * cols
    i = 0
    j = 0
    k = 0

    Up = True

    while k < resolution:
        if Up: 
            while j < rows and i >= 0:
                output.append(img[i, j])
                k += 1
                i -= 1
                j += 1


            if i < 0 and j <= rows - 1:
                i == 0
            if j == rows:
                i = i + 2
                j -= 1

        else:
            while i < cols and j >= 0:
                output.append(img[i, j])
                k += 1
                i += 1
                j -= 1

            if j < 0 and i <= cols - 1: 
                j = 0
            if i == cols: 
                j = j + 2
                i -= 1

    return output

In [35]:
# Generate a 9x9 matrix with random bytes (values between 0 and 255)
image_matrix = np.random.randint(0, 256, size=(9, 9), dtype=np.uint8)

print("9x9 Image Matrix (Simulated Bytes):")
print(image_matrix)

rows, cols = image_matrix.shape

print(serpentine(image_matrix))
            

9x9 Image Matrix (Simulated Bytes):
[[ 66 143 195 155 152 149  90 111 105]
 [189 121 213 231  72 114  63  56   7]
 [ 58 180 100  92 124   1 246  62 150]
 [193 226 155 139  97  85 238  15 128]
 [ 92 219 195  51 210   0 123 201 213]
 [177 225  10 203 250  77  78 252 159]
 [154 157  24 168 228 151 159 241  30]
 [ 66 161 134 130  14 192 177   2  19]
 [  0 101  73 205 148 122  90 129  51]]
[np.uint8(66), np.uint8(143), np.uint8(195), np.uint8(155), np.uint8(152), np.uint8(149), np.uint8(90), np.uint8(111), np.uint8(105), np.uint8(7), np.uint8(56), np.uint8(63), np.uint8(114), np.uint8(72), np.uint8(231), np.uint8(213), np.uint8(121), np.uint8(189), np.uint8(58), np.uint8(180), np.uint8(100), np.uint8(92), np.uint8(124), np.uint8(1), np.uint8(246), np.uint8(62), np.uint8(150), np.uint8(128), np.uint8(15), np.uint8(238), np.uint8(85), np.uint8(97), np.uint8(139), np.uint8(155), np.uint8(226), np.uint8(193), np.uint8(92), np.uint8(219), np.uint8(195), np.uint8(51), np.uint8(210), np.uint8(0), 

5) Use FFMPEG to transform the previous image into b/w. Do the hardest compression you can.

In [ ]:
ffmpeg.input('DSC01760.JPEG').output('DSC01760_GRAY.JPEG', vf='format=gray').run(overwrite_output=True)

In [ ]:
# Add everything into a new method and comment the results

Comment de results:

- 

5) Create a method which applies a run-lenght encoding from a series of bytes given.

In [ ]:
def runLengthEncoding(bytes):

    encoded=[]
    byte=bytes[0]

    for i in bytes:
        encoded.append((byte))
        byte = i

    encoded.append((byte))

    return encoded

In [ ]:
# Testing run length encoding

data = np.random.randint(0, 256, size=10, dtype=np.uint8)
print("Random Bytes:", data)

encoded_data = runLengthEncoding(data)
print("Encoded data:", encoded_data)


6) Create a class which can convert, can decode (or both) an input using the DCT. Not necessary a JPG encoder or decoder. A class only about DCT is OK too

In [ ]:
class DCTProcessor:
    def __init__(self, type=2, norm='ortho'):
        self.type = type
        self.norm = norm

    def encode(self, data):
        if data.ndim == 1:
            return dct(data, type=self.type, norm=self.norm)
        elif data.ndim == 2:
            return dct(dct(data.T, type=self.type, norm=self.norm).T, type=self.type, norm=self.norm)
        else:
            raise ValueError("Error")

    def decode(self, transformed_data):
        if transformed_data.ndim == 1:
            return idct(transformed_data, type=self.type, norm=self.norm)
        elif transformed_data.ndim == 2:
            return idct(idct(transformed_data.T, type=self.type, norm=self.norm).T, type=self.type, norm=self.norm)
        else:
            raise ValueError("Error")

    def process(self, data):
        encoded_data = self.encode(data)
        decoded_data = self.decode(encoded_data)
        return data, encoded_data, decoded_data

In [ ]:
if __name__ == "__main__":
    # Create an instance of the DCTProcessor
    dct_processor = DCTProcessor(type=2, norm='ortho')

    # Test with 1D data
    data_1d = np.array([12, 34, 54, 23, 65, 23, 45, 76, 23, 56])
    
    # Test with 2D data
    data_2d = np.array([[12, 34, 54, 23],
                        [65, 23, 45, 76],
                        [23, 56, 12, 89],
                        [13, 45, 67, 32]])

    # Process the 1D data
    original_1d, encoded_1d, decoded_1d = dct_processor.process(data_1d)
    print("1D Data:")
    print("Original:", original_1d)
    print("Encoded (DCT):", encoded_1d)
    print("Decoded (IDCT):", decoded_1d)

    print("\n")

    # Process the 2D data
    original_2d, encoded_2d, decoded_2d = dct_processor.process(data_2d)
    print("2D Data:")
    print("Original:")
    print(original_2d)
    print("Encoded (DCT):")
    print(encoded_2d)
    print("Decoded (IDCT):")
    print(decoded_2d)

7) Create a class which can convert, can decode (or both) an input using the DWT. Not necessary aJPEG2000 encoder or decoder. A class only about DWT is OK too

In [ ]:
class DWTProcessor:
    def __init__(self, wavelet='haar', level=1):
        self.wavelet = wavelet
        self.level = level

    def encode(self, data):
        if data.ndim == 1:
            coeffs = pywt.wavedec(data, wavelet=self.wavelet, level=self.level)
            return coeffs
        elif data.ndim == 2:
            coeffs = pywt.dwt2(data, wavelet=self.wavelet)
            return coeffs
        else:
            raise ValueError("Error")

    def decode(self, transformed_data):
        if isinstance(transformed_data, list):
            data = pywt.waverec(transformed_data, wavelet=self.wavelet)
            return data
        elif isinstance(transformed_data, tuple):
            data = pywt.idwt2(transformed_data, wavelet=self.wavelet)
            return data
        else:
            raise ValueError("Error")

    def process(self, data):
        encoded_data = self.encode(data)
        decoded_data = self.decode(encoded_data)
        return data, encoded_data, decoded_data

In [ ]:
if __name__ == "__main__":
    # Create an instance of the DWTProcessor
    dwt_processor = DWTProcessor(wavelet='haar', level=1)

    # Test with 1D data
    data_1d = np.array([12, 34, 54, 23, 65, 23, 45, 76, 23, 56])
    
    # Test with 2D data
    data_2d = np.array([[12, 34, 54, 23],
                        [65, 23, 45, 76],
                        [23, 56, 12, 89],
                        [13, 45, 67, 32]])

    # Process the 1D data
    original_1d, encoded_1d, decoded_1d = dwt_processor.process(data_1d)
    print("1D Data:")
    print("Original:", original_1d)
    print("Encoded (DWT):", encoded_1d)
    print("Decoded (IDWT):", decoded_1d)

    print("\n")

    # Process the 2D data
    original_2d, encoded_2d, decoded_2d = dwt_processor.process(data_2d)
    print("2D Data:")
    print("Original:")
    print(original_2d)
    print("Encoded (DWT):")
    print(encoded_2d)
    print("Decoded (IDWT):")
    print(decoded_2d)